In [1]:
import os
import configparser
import sys

import torch
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from scipy.optimize import minimize

# 設定の読み込み
config = configparser.ConfigParser()
config_path = "../config.ini"
config.read(config_path)
PROJECT_DIR = config["paths"]["project_dir"]
EXPT_RESULT_DIR = config["paths"]["results_dir"]
LOG_DIR = config["paths"]["logs_dir"]
sys.path.append(PROJECT_DIR)

from src.bnn import BayesianMLPModel
from src.utils_experiment import generate_integer_samples, negate_function

In [ ]:
def fit_pytorch_model_with_constraint(model, bounds, num_epochs=1000, learning_rate=0.01):
    def g(X, bounds):
        """
        制約：x1 == x2
        """
        X1 = X[:, 0]
        X2 = X[:, 1]

        return (X1 == X2).float().unsqueeze(1)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    for _ in range(num_epochs):
        optimizer.zero_grad()

        g_eval = g(model.train_inputs, bounds)
        loss = -(model(model.train_inputs).log_prob(model.train_targets).T @ g_eval) / model.train_targets.size(0)

        loss.backward()
        optimizer.step()